In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

#모델링
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import lightgbm as lgbm

#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error

In [10]:
# 파일 불러오기
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
#submission = pd.read_csv('test_submission.csv', index_col=0)

In [11]:
print(train.shape)
print(test.shape)
#print(submission.shape)

(36425, 77)
(9107, 76)


In [12]:
print(train.columns)

Index(['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE',
       'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE',
       'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE',
       'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE',
       'age_group', 'education', 'engnat', 'familysize', 'gender', 'hand',
       'married', 'race', 'religion', 'tp01', 'tp02', 'tp03', 'tp04', 'tp05',
       'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'voted', 'wf_01',
       'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06',
       'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13'],
      dtype='object')


In [13]:
train_x = train.drop('voted', axis = 1)
train_y = train['voted']

train_x.shape

(36425, 76)

In [14]:
print(train_x.race.value_counts())

White                    25042
Asian                     5466
Other                     3462
Black                     1708
Native American            434
Arab                       272
Indigenous Australian       41
Name: race, dtype: int64


In [15]:
# 원핫인코딩
print("columns list : ", list(train_x.columns), "\n")
train_x_dummies = pd.get_dummies(train_x)
print("get_dummies columns list : ", list(train_x_dummies.columns))

columns list :  ['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE', 'age_group', 'education', 'engnat', 'familysize', 'gender', 'hand', 'married', 'race', 'religion', 'tp01', 'tp02', 'tp03', 'tp04', 'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'wf_01', 'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13'] 

get_dummies columns list :  ['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE', 'education', 'engnat', 'familysize', 'hand', 'married', 'tp0

In [16]:
print("X: {}\ntrain_x_dummies: {}".format(train_x.shape, train_x_dummies.shape))

X: (36425, 76)
train_x_dummies: (36425, 100)


In [17]:
test = pd.get_dummies(test)
test.shape

(9107, 100)

In [18]:
print("True") if list(train_x_dummies.columns) == list(test.columns) else list(test.columns)

True


In [19]:
train_x_dummies

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
index,,,,,,,,,,,,,,,,,,,,,
43375,1.0,324,5.0,1067,5.0,359,1.0,709,1.0,742,...,0,1,0,0,0,0,0,0,0,0
4486,1.0,534,5.0,1555,5.0,2024,1.0,1097,1.0,3077,...,0,0,0,0,0,1,0,0,0,0
10262,5.0,609,1.0,749,2.0,624,1.0,1833,3.0,1474,...,0,0,0,1,0,0,0,0,0,0
14088,4.0,182,1.0,2969,1.0,1955,4.0,4630,1.0,1321,...,0,0,0,0,0,1,0,0,0,0
13398,4.0,549,5.0,1679,5.0,481,3.0,595,1.0,1202,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16511,1.0,312,5.0,965,5.0,499,1.0,1188,1.0,437,...,0,0,0,0,0,0,0,0,1,0
33507,3.0,474,4.0,1033,5.0,602,2.0,703,1.0,782,...,0,0,0,0,0,0,1,0,0,0
44917,2.0,617,4.0,1262,2.0,709,1.0,1063,2.0,1728,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#missing 확인
print('train : ',train_x_dummies.isnull().sum())
print('test : ',test.isnull().sum())

train :  QaA                0
QaE                0
QbA                0
QbE                0
QcA                0
                  ..
religion_Hindu     0
religion_Jewish    0
religion_Muslim    0
religion_Other     0
religion_Sikh      0
Length: 100, dtype: int64
test :  QaA                0
QaE                0
QbA                0
QbE                0
QcA                0
                  ..
religion_Hindu     0
religion_Jewish    0
religion_Muslim    0
religion_Other     0
religion_Sikh      0
Length: 100, dtype: int64


In [22]:
model = lgbm.LGBMClassifier(n_estimators=500)
model.fit(train_x_dummies, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [23]:
pred = model.predict(test)

In [24]:
#submission['voted']=pred
#submission.to_csv('test_submission.csv')